In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 32kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58f6c000 @  0x7f4dfa8c32a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 24.1MB/s 
    100% |████████████████████████████████| 2.0MB 4.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import  Variable


In [0]:
# Load dataset
# Make dataset iterable
# create model class
# Instantiate model class
# Instantiate loss class
# Instantiate optimizer class
# Train model

In [4]:
!wget https://dl.dropboxusercontent.com/s/b3khjfddz5wpvai/alzhdset.zip?dl=0 -O alzhdset.zip;

--2018-10-17 15:06:07--  https://dl.dropboxusercontent.com/s/b3khjfddz5wpvai/alzhdset.zip?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.7.6, 2620:100:601b:6::a27d:806
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.7.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35988843 (34M) [application/zip]
Saving to: ‘alzhdset.zip’

alzhdset.zip        100%[===================>]  34.32M  18.6MB/s    in 1.8s    

2018-10-17 15:06:11 (18.6 MB/s) - ‘alzhdset.zip’ saved [35988843/35988843]



In [0]:
# !apt-get install fastjar

In [4]:
!ls

alzhdset  alzhdset.zip	sample_data


In [7]:
!unzip alzhdset.zip

Archive:  alzhdset.zip
   creating: Alzheimer's Dataset/
   creating: Alzheimer's Dataset/Train/
   creating: Alzheimer's Dataset/Train/MildDemented/
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (10).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (11).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (12).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (13).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (14).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (15).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (16).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (17).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (18).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (19).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (2).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (20).jpg  
  inflating: Alzheimer's Dataset/Train/MildDemented/01 (21)

In [0]:
!mv "Alzheimer's Dataset" alzhdset

In [0]:
trans = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [0]:
train_data = dsets.ImageFolder("./alzhdset/Train",transform=trans)
test_data = dsets.ImageFolder("./alzhdset/Test",transform=trans)

In [0]:
# Make dataset iterable


batch_size = 64
n_iters = 3000
num_epochs = int(n_iters / (5121/batch_size))

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [0]:
# Conv -> Max pool -> Conv -> Max pool -> FC

In [0]:
# Create Model Class
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    
    # Convolution 1
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.relu1 = nn.ReLU()
    
    # Max pool 1
    self.maxpool1 = nn.AdaptiveMaxPool2d(output_size=14)
    
    # Convolution 2
    self.cnn2 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.relu2 = nn.ReLU()
    
    # Max pool 2
    self.maxpool2 = nn.AdaptiveMaxPool2d(output_size=7)
    
    # FC (readout)
    self.fc1 = nn.Linear(32*7*7, 4)
   
    
  def forward(self, x):
    # Convolution 1
    out = self.cnn1(x)
    out = self.relu1(out)
    
    # Max pool 1
    out = self.maxpool1(out)
    
    # Convolution 2
    out = self.cnn2(x)
    out = self.relu2(out)
    
    # Max pool 2
    out = self.maxpool2(out)
    
    # Resize
    # 100 is the batch size...
    # (100, 32, 7, 7) -> (100, 32*7*7)
    out = out.view(out.size(0), -1)
    
    # Linear (readout)
    out = self.fc1(out)
    
    return out

In [52]:
model = CNN()
model.cuda()

CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): AdaptiveMaxPool2d(output_size=14)
  (cnn2): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): AdaptiveMaxPool2d(output_size=7)
  (fc1): Linear(in_features=1568, out_features=4, bias=True)
)

In [0]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [54]:
# Train the model

# Convert input/labels to Variables
# Clear gradient buffers
# Get output given the inputs
# Get loss
# Get gradients w.r.t parameters
# Update parameters using the gradients
# Repeat

import time
tick = time.time()

iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    
    
    images = Variable(images.cuda()) # No need to resize
    labels = Variable(labels.cuda())
    
    optimizer.zero_grad()
    
    outputs = model(images)
    
    loss = criterion(outputs, labels)
    
    loss.backward()
    
    optimizer.step()
    
    iter += 1
    
    if iter % 500 == 0:
      correct = 0
      total = 0
      
      for images, labels in test_loader:
        
        images = Variable(images.cuda())
        
        outputs = model(images.cuda())
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += ((predicted.cpu()) == (labels.cpu())).sum().item()
        
      accuracy = 100 * correct / total
      
      print('Iteration: {}, Loss: {}, Accuracy:{}'.format(iter, loss.data[0], accuracy))
    
tock = time.time()    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration: 500, Loss: 0.9553308486938477, Accuracy:51.83737294761532
Iteration: 1000, Loss: 0.9312585592269897, Accuracy:53.87021110242377
Iteration: 1500, Loss: 0.7760425209999084, Accuracy:55.12118842845973
Iteration: 2000, Loss: 0.8502954244613647, Accuracy:55.90304925723221
Iteration: 2500, Loss: 0.7782031297683716, Accuracy:54.26114151681001


In [55]:
print('Time required is', tock - tick)

Time required is 277.5077095031738
